This is the Kmeans clustering part of our model

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt

In [55]:
## here is going to be the code for reading and getting the data that we want
all_mostrecent = pd.read_csv("Most-Recent-Cohorts-Institution.csv")
filtered_mostrecent = all_mostrecent[["UNITID", "INSTNM", "CITY","STABBR","CONTROL","ADM_RATE", "SATVRMID","SATMTMID","ACTCMMID","ACTCM25",
        "ACTCM75", "COSTT4_A","UGDS","C150_4","RET_FT4","MD_EARN_WNE_P6",
        "MD_EARN_WNE_P10", "MD_EARN_WNE_MALE0_P6","MD_EARN_WNE_MALE1_P6",
        "MD_EARN_WNE_MALE0_P10","MD_EARN_WNE_MALE1_P10", "FTFTPCTFLOAN","FTFTPCTPELL", 
        "DEBT_MDN", "CDR3", "LOCALE", "LATITUDE", "LONGITUDE"]]
no_na_mostrecent = filtered_mostrecent.dropna(axis=0, inplace = False)
## The following two are the good tables.
numerical_mostrecent = no_na_mostrecent.select_dtypes(include=['number']).drop('UNITID', axis = 1)
numerical_mostrecent["CONTROL"].replace(3,2, inplace=True)
unitid_and_state = no_na_mostrecent[['UNITID', "INSTNM"]]


/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (9,1537,1540,1542,1606,1608,1614,1615,1619,1620,1621,1622,1623,1624,1625,1626,1627,1628,1629,1703,1704,1725,1726,1727,1728,1729,1743,1815,1816,1817,1818,1823,1824,1830,1831,1879,1880,1881,1882,1883,1884,1885,1886,1887,1888,1889,1890,1891,1892,1893,1894,1895,1896,1897,1898,1909,1910,1911,1912,1913,1957,1958,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1983,1984,2376,2377,2403,2404,2495,2496,2497,2498,2499,2500,2501,2502,2503,2504,2505,2506,2507,2508,2509,2510,2511,2512,2513,2514,2515,2516,2517,2518,2519,2520,2521,2522,2523,2524,2525,2526,2527,2528,2529,2530,2958) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [105]:
## The point of this function is too assign weights give an list of user preferences
print(numerical_mostrecent.columns)
def assign_weights(preferences):
    ##features to include:
    ## Control, Cost_4, UGDS, MDEARN_P6, LATITUDE, LONGITUDE
    weights = [.2] * 23
    weights[21] = 10
    weights[22] = 10
    return weights
x = assign_weights([])
print(x)

Index(['CONTROL', 'ADM_RATE', 'SATVRMID', 'SATMTMID', 'ACTCMMID', 'ACTCM25',
       'ACTCM75', 'COSTT4_A', 'UGDS', 'C150_4', 'RET_FT4', 'MD_EARN_WNE_P6',
       'MD_EARN_WNE_P10', 'MD_EARN_WNE_MALE0_P6', 'MD_EARN_WNE_MALE1_P6',
       'MD_EARN_WNE_MALE0_P10', 'MD_EARN_WNE_MALE1_P10', 'FTFTPCTFLOAN',
       'FTFTPCTPELL', 'CDR3', 'LOCALE', 'LATITUDE', 'LONGITUDE'],
      dtype='object')
[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 10, 10]


In [86]:
## This is the cell where we normalize the numerical dataset
min_value = 0
max_value = 1
normalized_df = (numerical_mostrecent - numerical_mostrecent.min()) / (numerical_mostrecent.max() - numerical_mostrecent.min()) * (max_value - min_value) + min_value
normalized_df.reset_index(inplace=True)
normalized_df = normalized_df.drop('index', axis = 1)
unitid_and_state.reset_index(inplace=True)
unitid_and_state = unitid_and_state.drop('index', axis = 1)
##normalized df contains the data that we want

In [87]:
normalized_df

,CONTROL,ADM_RATE,SATVRMID,SATMTMID,ACTCMMID,ACTCM25,ACTCM75,COSTT4_A,UGDS,C150_4,...,MD_EARN_WNE_MALE0_P6,MD_EARN_WNE_MALE1_P6,MD_EARN_WNE_MALE0_P10,MD_EARN_WNE_MALE1_P10,FTFTPCTFLOAN,FTFTPCTPELL,CDR3,LOCALE,LATITUDE,LONGITUDE
0,0.0,0.704321,0.219178,0.166667,0.190476,0.173913,0.238095,0.146911,0.069447,0.236096,...,0.066490,0.056870,0.173308,0.070341,0.583572,0.651848,0.319149,0.03125,0.353429,0.767997
1,0.0,0.880687,0.663014,0.638462,0.619048,0.521739,0.714286,0.208796,0.182920,0.613981,...,0.199413,0.198118,0.252679,0.206708,0.446948,0.288472,0.085106,0.03125,0.325960,0.765517
2,0.0,0.725872,0.671233,0.666667,0.666667,0.565217,0.761905,0.189060,0.100775,0.594966,...,0.251560,0.297877,0.264293,0.266301,0.371467,0.186100,0.113475,0.03125,0.352165,0.767224
3,0.0,0.979073,0.254795,0.158974,0.190476,0.173913,0.238095,0.147112,0.047227,0.240053,...,0.052837,0.025479,0.117846,0.046671,0.657108,0.728185,0.368794,0.03125,0.301421,0.770928
4,0.0,0.780323,0.589041,0.551282,0.571429,0.434783,0.761905,0.277522,0.424682,0.721477,...,0.239947,0.257283,0.282311,0.254531,0.352212,0.121125,0.085106,0.03125,0.319643,0.757496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875,0.0,0.643311,0.712329,0.641026,0.714286,0.695652,0.714286,0.083038,0.061297,0.717191,...,0.288770,0.325088,0.407648,0.351805,0.074150,0.112079,0.063830,0.03125,0.243053,0.813426
876,0.0,0.808329,0.506849,0.448718,0.428571,0.391304,0.523810,0.188903,0.521964,0.461530,...,0.209287,0.225977,0.269155,0.236004,0.382835,0.353558,0.127660,0.31250,0.337422,0.789327
877,1.0,0.587194,0.402740,0.400000,0.523810,0.434783,0.666667,0.504351,0.009454,0.697516,...,0.211697,0.170302,0.195789,0.169257,0.534515,0.167678,0.156028,0.34375,0.386039,0.813918
878,0.0,0.753878,0.424658,0.371795,0.428571,0.347826,0.523810,0.052782,0.087440,0.370301,...,0.184457,0.203330,0.179786,0.194765,0.326403,0.343188,0.156028,0.31250,0.499644,0.758267


In [88]:
def euclidean_distance(row1, row2):
    return np.linalg.norm(x * (row1 - row2))

In [89]:
def closest_schools(unitid, number):
    rownum = unitid_and_state['UNITID'].index[unitid_and_state['UNITID'] == unitid][0]
    target_row = normalized_df.iloc[rownum]
    
    # Find the Euclidean distances
    result = normalized_df.drop(rownum)
    distances = [euclidean_distance(target_row, row) for _, row in result.iterrows()]

    # Get the indices of the smallest 10 distances
    min_distance_indices = np.argsort(distances)[:number]
    min_distances = [distances[i] for i in min_distance_indices]

    # Get the rows with the smallest 10 distances
    rows_with_min_distances = result.iloc[min_distance_indices]

    
    print("Closest Schools")
    return unitid_and_state.iloc[min_distance_indices]
    
    

In [106]:
closest_schools(152080, 10)

Closest Schools


,UNITID,INSTNM
161,144050,University of Chicago
183,147767,Northwestern University
172,145725,Illinois Institute of Technology
178,146719,Loyola University Chicago
344,170301,Hope College
176,146481,Lake Forest College
206,150774,Holy Cross College
195,149781,Wheaton College
333,168528,Adrian College
203,150668,Goshen College
